# 参数管理
本节包含以下内容：

* 访问参数，用与调试、诊断和可视化
* 参数初始化；
* 在不同模型组件间共享参数。

**（首先先看一下具有单隐藏层的多层感知机）**

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0151],
        [ 0.1832]], grad_fn=<AddmmBackward0>)

## 参数访问
我们可以从已有模型中访问参数，当通过`Sequential`类定义模型时，我们可以通过索引来访问模型的任意层。就像模型是一个列表一样，每一层的 参数都在其中。下面的代码是访问第二个全连接层的参数。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0397,  0.2020,  0.3093, -0.3450,  0.2808,  0.2752,  0.1928,  0.0710]])), ('bias', tensor([-0.1989]))])


输出的结果告诉我们一些事情。首先，这个全连接层包含两个参数，分别是该层的权重和偏置。两者都存储为单精度浮点数。参数名称允许唯一标识每一个参数，即使包含在数百层的网络中也是如此。

## 目标参数
每个参数都表示为参数类的实例。要对参数执行任何操作，首先我们需要访问底层的数值。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1989], requires_grad=True)
tensor([-0.1989])


参数是复合对象，包含值、梯度和额外信息。这就是我们需要显式参数值的原因。除了值之外，我们还可以访问每个参数的梯度。在这个上面的网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [4]:
net[2].weight.grad == None

True

## 一次性访问所有的参数
递归整个树来提取每个子块的参数

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示：

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.1989])

## 从嵌套块收集参数
首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[0.3206],
        [0.3206]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表去索引他们。下面访问第一个主要的块中、第二个子块的第一层的偏置项。

In [9]:
rgnet[0][1][0].bias.data

tensor([-0.0007, -0.2213,  0.2844,  0.0139, -0.1214, -0.0520, -0.2354, -0.0990])

## 参数初始化
默认情况下，Pytorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算的。Pytorch的`nn.init`模块提供了多种预置初始化方法

### 内置初始化
调用内置初始化器。下面的代码将所有的权重参数初始化为标准差为0.01的该死随机变量，且将偏执参数设置为0.

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0160, -0.0084,  0.0005, -0.0014]), tensor(0.))

还可以将所有的参数初始化为给定的常数，比如初始化为1。

In [11]:
def init_condtant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_condtant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以 **[对某些块应用不同的初始化方法]**。

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0985,  0.3780,  0.3214, -0.0392])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化
在下面的例子中，我们将使用一下的分布为任何权重参数 $w$ 定义初始化方法：
$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$
同样，实现了一个`my_init`函数来应用到`net`。

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -7.2125,  0.0000, -0.0000],
        [ 5.4764,  0.0000, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

## 参数绑定


有时，我们希望在多个层间共享参数，我们可以定义一个稠密层，然后用他的参数来设置另一个层的参数。

In [17]:
# 给共享层一个名称，以便以后可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保他们实际上是同一对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
